# ipyautoui

wrapper that sits on top of ipywidgets and other ipy widget libraries to template / automate the creation of widget forms. Uses pydantic to create defined data-container and serialisation to JSON. Includes example patterns for adding new custom widgets.

## Create a UI object

ipyautoui creates a row of widgets with a: `name`, `value` and `label`. The `value` is interpreted by the WidgetRowBase class and a widget is widget type is guess by the type of the value and the `kwargs` that can also be passed when initialising the widget.

In [ ]:
# TODO: refer to ipyfilechooser for inspiration for README.md

In [1]:
import sys
sys.path.append('src')
from ipyautoui.autoui import AutoUi

?AutoUi

Init signature:
AutoUi(
    data: Dict,
    config_autoui: ipyautoui.autoui.AutoUiConfig = None,
    path: pathlib.Path = None,
)
Docstring:     
AutoUi widget. generates UI form from pydantic schema. keeps the "value" field
up-to-date on_change 

Args:
    traitlets.HasTraits ([type]): traitlets.HasTraits makes it possible to observe the value
        of this widget.
Init docstring:
init AutoUi

Args:
    pydantic_obj (typing.Type[BaseModel]): initiated pydantic data object
    di_widgets_mapper (typing.Dict, optional): [description]. Defaults to DI_WIDGETS_MAPPER.
        if new mappings given they extend DI_WIDGETS_MAPPER. it is expected that renaming 
        schema keys (call_rename_schema_keys) is done in the filter function within di_widgets_mapper
File:           /mnt/c/engDev/git_mf/ipyautoui/src/ipyautoui/autoui.py
Type:           MetaHasTraits
Subclasses:     


In [2]:
rows = [
    {'value': 1, 'name':'integer'},
    {'value': 'string'}, 
    {'value': 1, 'kwargs': {'min':0, 'max':4}}
]

aui = AutoUiBase(rows = rows)
ui = AutoUi(aui)
ui

In [3]:
# AutoUi class adds the widget attribute
# it uses traitlets to ensure that if the widget value
# changes the ui.value attribute also changes with it
display(ui.rows[0].widget)
print(ui.rows[0].widget.value)
ui.rows[0].widget.value = 2
ui.rows[0].value

IntText(value=1)

1


2

## Development installation

For a development installation (requires JupyterLab (version >= 3), yarn, and mamba.):

```
$ git clone https://github.com/jgunstone/ipyautoui
$ cd ipyautoui
$ mamba env create --file environment-dev.yml
```


## Output / transport data from UI

In [4]:
## using Pydantic data serialisation amd config option introduced in v1.8.3,
# the JSON inconpatible attributes are removed
# when the widget is written to dict or JSON. 
ui.dict()

{'rows': [{'value': 2,
   'kwargs': {},
   'autoui_type': "<class 'ipywidgets.widgets.widget_int.IntText'>",
   'name': 'integer',
   'label': 'label'},
  {'value': 'string',
   'kwargs': {},
   'autoui_type': "<class 'ipywidgets.widgets.widget_string.Text'>",
   'name': 'name',
   'label': 'label'},
  {'value': 1,
   'kwargs': {'min': 0, 'max': 4},
   'autoui_type': "<class 'ipywidgets.widgets.widget_int.IntSlider'>",
   'name': 'name',
   'label': 'label'}]}

## Reload data back into UI

the `autoui_type` saves a class string defining what widget should be used. This is converted to the object using importlib.

In [5]:
#  importlib is used to create the widget from the `autoui_type` string characteristic
aui = AutoUiBase(**ui.dict())
AutoUi(aui)

## Create custom UI object rows

This library mainly relies on ipywidgets, but other libraries can also be used. The AutoUi widget uses traitlets to watch when the `widget.value` changes, wrappers can be made around other widget libraries to allow this behaviour. 

In this way pandas dataframes using ipydatagrid can also be used.

(See `ipyautoui.custom_widgets` for more examples.

In [7]:
import pandas as pd
rows.append(
    {'name':'pandas', 'value': pd.DataFrame({'col1': [1, 2], 'col2': [3, 4]}), 'label': 'example of a custom widget'}
)

In [8]:
aui = AutoUiBase(rows = rows)
ui = AutoUi(aui)
ui

## Write to file